In [ ]:
import yfinance as yf
import pandas as pd
import psycopg as pg
from daily_insert.config import DB_CONFIG

with pg.connect(**DB_CONFIG) as conn:
    with conn.cursor() as cur:
        cur.execute('SELECT ticker FROM tickers')
        tickers = cur.fetchall()

assets = [ticker[0] for ticker in tickers]

df_list = []
for asset in assets:
    data = yf.download(asset, interval='5m', auto_adjust=False)
    data = data.droplevel(1, axis=1)
    data['ticker'] = asset
    df_list.append(data)

df = pd.concat(df_list, axis=0)

# Data cleaning
mask = (df['High'] < df['Open']) | (df['High'] < df['Close']) | (df['Low'] > df['Open']) | (df['Low'] > df['Close'])
clean = df[~mask].copy()
temp = df[mask].copy()

temp['High'] = temp[['Open', 'Close', 'High']].max(axis=1)
temp['Low'] = temp[['Open', 'Close', 'Low']].min(axis=1)
clean = pd.concat([clean, temp], axis=0)
clean = clean.reset_index()
clean['Volume'] = clean['Volume'].astype(int)

print(f'Rows to be inserted: {len(clean)}')

from io import StringIO

# Create buffer
buffer = StringIO()
clean.to_csv(buffer, index=False, header=False)
buffer.seek(0)


with pg.connect(**DB_CONFIG) as conn:
    # The COPY operation needs to be within a WITH block
    with conn.cursor().copy("COPY five_minute (date, adj_close, close, high, low, open, volume, ticker) FROM STDIN WITH CSV") as copy:
        copy.write(buffer.getvalue())
    
    conn.commit()

[*********************100%***********************]  1 of 1 completed

1 Failed download:
['AAF.L']: YFPricesMissingError('$%ticker%: possibly delisted; no price data found  (5m 2024-12-15 14:24:24+00:00 -> 2025-02-13 14:24:24+00:00) (Yahoo error = "5m data not available for startTime=1734272664 and endTime=1739456664. The requested range must be within the last 60 days.")')
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

1 Failed download:
['ABF.L']: YFPricesMissingError('$%ticker%: possibly delisted; no price data found  (5m 2024-12-15 14:24:27+00:00 -> 2025-02-13 14:24:27+00:00) (Yahoo error = "5m data not available for startTime=1734272667 and endTime=1739456667. The requested range must be within the last 60 days.")')
[*********************100%*************

Rows to be inserted: 746856


DatetimeFieldOverflow: date/time field value out of range: "2468.25146484375"
CONTEXT:  COPY five_minute, line 1, column date: "2468.25146484375"